In [1]:
import os
import github as gh
import bs4 as bs
import requests
from lxml import etree

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
from tqdm import tqdm

In [2]:
GITHUB_ACCESS_TOKEN=os.getenv('GITHUB_ACCESS_TOKEN')
g=gh.Github(GITHUB_ACCESS_TOKEN)
print(g.get_user())

AuthenticatedUser(login=None)


In [3]:
query='tensorflow language:python created:2021-04-01..2021-04-02'
urls = []
result=g.search_repositories(query)
print(result.totalCount)
print(dir(result))
for repository in result:
    urls.append(repository.html_url)
    

29
['_PaginatedListBase__elements', '_PaginatedListBase__fetchToIndex', '_PaginatedList__contentClass', '_PaginatedList__firstParams', '_PaginatedList__firstUrl', '_PaginatedList__headers', '_PaginatedList__list_item', '_PaginatedList__nextParams', '_PaginatedList__nextUrl', '_PaginatedList__parseLinkHeader', '_PaginatedList__requester', '_PaginatedList__reverse', '_PaginatedList__totalCount', '_Slice', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_couldGrow', '_fetchNextPage', '_getLastPageUrl', '_grow', '_isBiggerThan', '_reversed', 'get_page', 'reversed', 'totalCount']


In [4]:
url='https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras'
url2='https://github.com/bamblebam/image-classification-rps'

In [5]:
options=Options()
options.headless=True
driver=webdriver.Chrome(ChromeDriverManager(path='./').install(),options=options)
driver.get(url2)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [./\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [6]:
table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")

In [7]:
# links=table.find_elements_by_tag_name('a')
# print(len(links))
# for link in links:
#     href=link.get_attribute('href')
#     if 'tree' in href or '.py' in href:
#         stack.append(href)

# driver.get(url2)
# table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
# links=table.find_elements_by_tag_name('a')
# for link in links:
#     href=link.get_attribute('href')
#     if '/tree/' in href or '.py' in href:
#         stack.append(href)

In [29]:
queue=list()
full_list=list()
links_list = set()
sequential_list=list()

In [17]:
def push_to_queue(links):
    for link in links:
        href=link.get_attribute('href')
        if href in links_list:
            continue
        if '/tree/' in href or '.py' in href:
            links_list.add(href)
            queue.append(href)

In [ ]:
# def if_sequential():
#     code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
#     if 'Sequential' in code_body.text:
#             count+=1
#     return count

In [18]:
def search_through_files(link):
    if '/tree/' in link:
        driver.get(link)
        time.sleep(2.5)
        table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-block']")
        links=table.find_elements_by_tag_name('a')
        push_to_queue(links)
    elif '.py' in link:
        driver.get(link)
        time.sleep(2.5)
        full_list.append(link)
        code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
        if 'Sequential' in code_body.text:
            sequential_list.append(url) 

In [19]:
def bfs():
    while queue:
        link=queue.pop(0)
        search_through_files(link)

In [20]:
def get_all_relevant_links(url):
    links_list.add(url)
    driver.get(url)
    table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
    links=table.find_elements_by_tag_name('a')
    for link in links:
        href=link.get_attribute('href')
        if '/tree/' in href or '.py' in href:
            links_list.add(href)
            queue.append(href)
    bfs()
    print("Ended")

In [21]:
res=get_all_relevant_links('https://github.com/Z-yq/TensorflowTTS')

Ended


In [14]:
for i in full_list:
    print(i)

https://github.com/Z-yq/TensorflowTTS/blob/main/fastspeech_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/run-test.py
https://github.com/Z-yq/TensorflowTTS/blob/main/tacotron_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_acoustic.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_vocoder.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/fastspeech_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/tacotron_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/vocoder_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/__init__.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/conformer.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/fastspeech.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/model.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/positional_encoding.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/sw

In [26]:
def getSequentialModel(full_list):
    count=0
    for url in full_list:
        driver.get(url)
        time.sleep(2.5)
        print(url)
        code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
        if 'Sequential' in code_body.text:
            count+=1
    return count

In [27]:
files_with_sequential=getSequentialModel(full_list)


https://github.com/Z-yq/TensorflowTTS/blob/main/fastspeech_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/run-test.py
https://github.com/Z-yq/TensorflowTTS/blob/main/tacotron_extract_features.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_acoustic.py
https://github.com/Z-yq/TensorflowTTS/blob/main/train_vocoder.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/fastspeech_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/tacotron_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/dataloaders/vocoder_dataloader.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/__init__.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/conformer.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/fastspeech.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/model.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/positional_encoding.py
https://github.com/Z-yq/TensorflowTTS/blob/main/models/sw

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']"}
  (Session info: headless chrome=91.0.4472.124)
